# Informe interactivo: empresas de turismo certificadas

Este cuaderno explora los datasets procesados en `data/processed` para entregar una visión ejecutiva de la oferta turística certificada en Chile. Todas las visualizaciones son interactivas y están construidas con Plotly.

## 1. Configuración
Cargamos las librerías clave y definimos la ruta base hacia los datos procesados.

In [1]:
# Configuración básica de análisis
from pathlib import Path
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

PROJECT_ROOT = Path.cwd()
if not (PROJECT_ROOT / 'data' / 'processed').exists():
    PROJECT_ROOT = PROJECT_ROOT.parent
DATA_DIR = PROJECT_ROOT / 'data' / 'processed'

pd.options.display.float_format = '{:,.1f}'.format

## 2. Carga y preparación de datos
Exploramos el dataset consolidado `tourism_metrics.csv` para derivar subconjuntos relevantes para alojamientos, guías y unidades habitacionales.

In [2]:
# Carga de datasets procesados
tourism_metrics = pd.read_csv(DATA_DIR / 'tourism_metrics.csv')

translation_map = {
    'Bed and Breakfast, Alojamiento Familiar u Hospedaje Rural': 'Cama y desayuno / alojamiento familiar',
    'Complejo Turístico o Resort': 'Complejo turístico o centro vacacional',
    'Centro de Turismo de Naturaleza o Lodge': 'Centro de turismo de naturaleza o ecoalbergue',
    'Apart - Hotel': 'Apartotel',
    'Hostel o Albergue (Refugio)': 'Albergue / refugio',
    'Hotel Boutique': 'Hotel boutique'
}

def apply_translation(df, column='category'):
    if column in df.columns:
        df[column] = df[column].replace(translation_map)
    return df

apply_translation(tourism_metrics)

lodging = apply_translation(
    tourism_metrics.query("metric_group == 'Tipos de Alojamientos'").copy()
)
guides = apply_translation(
    tourism_metrics.query("metric_group == 'Tipos de Clase Guías'").copy()
)
housing = tourism_metrics.query("metric_group == 'Unidades Habitacionales'").copy()
regions = tourism_metrics.query("metric_group == 'Regiones'").copy()

lodging_sorted = lodging.sort_values('value', ascending=False).copy()
lodging_sorted['cumulative_value'] = lodging_sorted['value'].cumsum()
lodging_sorted['cumulative_share'] = lodging_sorted['cumulative_value'] / lodging_sorted['value'].sum()
lodging_sorted['highlight_ratio'] = (
    lodging_sorted['highlight_value'].fillna(0) / lodging_sorted['value'].replace(0, pd.NA)
).fillna(0)

TOP_N = 10
lodging_top = lodging_sorted.head(TOP_N).copy()

summary = pd.DataFrame({
    'Indicador': [
        'Empresas certificadas (regiones)',
        'Unidades habitacionales certificadas',
        'Tipos de alojamientos monitoreados',
        'Clases de guías monitoreadas'
    ],
    'Valor': [
        regions.value.sum(),
        housing.value.sum(),
        lodging.category.nunique(),
        guides.category.nunique()
    ]
})
summary

,Indicador,Valor
0,Empresas certificadas (regiones),610.0
1,Unidades habitacionales certificadas,"17,593.0"
2,Tipos de alojamientos monitoreados,13.0
3,Clases de guías monitoreadas,3.0


## 3. Indicadores ejecutivos
Un vistazo rápido a los volúmenes críticos para la toma de decisiones: empresas, unidades habitacionales, masa de alojamientos certificados y nivel de guías especializados.

In [3]:
# Indicadores interactivos
highlight_rate = (lodging['highlight_value'].sum() / lodging['value'].sum()) * 100
kpi_data = [
    {"title": "Empresas certificadas", "value": regions['value'].sum(), "format": ",.0f", "suffix": ""},
    {"title": "Unidades habitacionales", "value": housing['value'].sum(), "format": ",.0f", "suffix": ""},
    {"title": "Registros de alojamientos", "value": lodging['value'].sum(), "format": ",.0f", "suffix": ""},
    {"title": "Casos destacados / total", "value": highlight_rate, "format": ".1f", "suffix": "%"}
]
fig_kpi = make_subplots(rows=1, cols=len(kpi_data), specs=[[{'type': 'indicator'}] * len(kpi_data)])
for idx, metric in enumerate(kpi_data, start=1):
    fig_kpi.add_trace(
        go.Indicator(
            mode='number',
            value=metric['value'],
            number={'valueformat': metric['format'], 'suffix': metric['suffix']},
            title={'text': metric['title']}
        ),
        row=1, col=idx
    )
fig_kpi.update_layout(height=250, margin=dict(l=0, r=0, t=40, b=0))
fig_kpi.show()

## 4. Oferta de alojamientos certificados
Analizamos la mezcla de alojamientos desde distintos ángulos: ranking por volumen, participación porcentual y el grado de concentración acumulada.

In [4]:
# Distribución de tipos de alojamientos certificados
fig = px.bar(
    lodging_sorted,
    x='value',
    y='category',
    color='percentage',
    color_continuous_scale='Blues',
    title='Cantidad de alojamientos certificados por tipo',
    labels={
        'value': 'Empresas certificadas',
        'category': 'Tipo de alojamiento',
        'percentage': 'Participación (%)'
    },
    text='value',
    custom_data=['percentage']
)
fig.update_traces(
    texttemplate='%{text:,.0f}',
    textposition='outside',
    hovertemplate=(
        '<b>%{y}</b><br>'
        'Empresas certificadas: %{x:,.0f}<br>'
        'Participación: %{customdata[0]:.1%}<extra></extra>'
    )
)
fig.update_layout(
    yaxis={'categoryorder': 'total ascending', 'title': 'Tipo de alojamiento'},
    xaxis_title='Empresas certificadas',
    coloraxis_colorbar=dict(title='Participación (%)'),
    height=600
)
fig.show()

In [5]:
# Exportar gráfica a HTML autonoma
output_html = PROJECT_ROOT / 'docs' / 'distribucion_alojamientos.html'
fig.write_html(output_html, include_plotlyjs='cdn', full_html=True)
print(f'Gráfica exportada a {output_html.resolve()}')

Gráfica exportada a /Users/brunosanmartinnavarro/Documents/UACh/empresas_certificadas/docs/distribucion_alojamientos.html


### 4.1 Distribución porcentual de las 10 tipologías líderes
El diagrama tipo dona permite comparar rápidamente el peso relativo de las tipologías con mayor masa crítica sobre el total certificado.

In [34]:
# Diagrama de dona para la participación porcentual
pie_fig = px.pie(
    lodging_top,
    names='category',
    values='value',
    hole=0.45,
    color='category',
    title='Participación de las principales tipologías de alojamiento'
 )
pie_fig.update_traces(
    textposition='inside',
    textinfo='percent+label',
    hovertemplate=(
        '<b>%{label}</b><br>'
        'Empresas certificadas: %{value:,.0f}<br>'
        'Participación: %{percent:.1%}<extra></extra>'
    )
)
pie_fig.update_layout(showlegend=False)
pie_fig.show()

### 4.2 Curva de Pareto
Cuantificamos qué proporción del portafolio queda cubierta a medida que se suman nuevas tipologías; útil para priorizar esfuerzos comerciales o de fiscalización.

In [35]:
# Curva de Pareto para tipologías de alojamiento
pareto_fig = make_subplots(specs=[[{"secondary_y": True}]])
pareto_fig.add_bar(
    x=lodging_sorted['category'],
    y=lodging_sorted['value'],
    name='Empresas certificadas',
    hovertemplate='<b>%{x}</b><br>Empresas certificadas: %{y:,.0f}<extra></extra>'
 )
pareto_fig.add_scatter(
    x=lodging_sorted['category'],
    y=lodging_sorted['cumulative_share'] * 100,
    mode='lines+markers',
    name='Participación acumulada',
    marker=dict(color='firebrick'),
    secondary_y=True,
    hovertemplate='<b>%{x}</b><br>Participación acumulada: %{y:.1f}%<extra></extra>'
 )
pareto_fig.update_layout(
    title='Curva de Pareto: empresas certificadas vs. cobertura acumulada',
    xaxis_title='Tipo de alojamiento',
    hovermode='x unified',
    height=500
 )
pareto_fig.update_yaxes(title_text='Empresas certificadas', secondary_y=False)
pareto_fig.update_yaxes(title_text='Participación acumulada (%)', range=[0, 110], secondary_y=True)
pareto_fig.show()

## 5. Contraste de casos destacados vs. oferta total
Relacionamos los casos destacados (`highlight_value`) con el volumen total para priorizar segmentos.

In [36]:
# Contraste entre oferta total y casos destacados
highlight_fig = px.scatter(
    lodging_sorted,
    x='value',
    y='highlight_value',
    size='value',
    hover_name='category',
    color='category',
    title='Casos destacados vs. total por tipo de alojamiento',
    custom_data=['percentage', 'highlight_percentage']
)
highlight_fig.update_traces(
    hovertemplate=(
        '<b>%{hovertext}</b><br>'
        'Empresas certificadas: %{x:,.0f}<br>'
        'Casos destacados: %{y:,.0f}<br>'
        'Participación total: %{customdata[0]:.1%}<br>'
        'Participación de destacados: %{customdata[1]:.2%}<extra></extra>'
    )
)
highlight_fig.update_layout(
    xaxis_title='Total certificado',
    yaxis_title='Casos destacados',
    legend_title='Tipo de alojamiento'
)
highlight_fig.show()

## 6. Clases de guías certificados
Identificamos dónde se concentra el capital humano certificado.

In [37]:
# Distribución de clases de guías certificados
guide_fig = px.bar(
    guides.sort_values('value', ascending=True),
    x='value',
    y='category',
    text='percentage',
    labels={'value': 'Guías certificados', 'category': 'Clase', 'percentage': 'Participación'},
    title='Clases de guías certificados',
    custom_data=['percentage']
)
guide_fig.update_traces(
    texttemplate='%{text:.1%}',
    textposition='outside',
    hovertemplate=(
        '<b>%{y}</b><br>'
        'Guías certificados: %{x:,.0f}<br>'
        'Participación: %{customdata[0]:.1%}<extra></extra>'
    )
)
guide_fig.update_layout(height=400)
guide_fig.show()

## 7. Vista jerárquica consolidada
Visualizamos el peso relativo de cada métrica dentro del portafolio certificado.

In [38]:
# Relación jerárquica de métricas de turismo certificado
sunburst_fig = px.sunburst(
    tourism_metrics,
    path=['metric_group', 'category'],
    values='value',
    color='metric_group',
    title='Peso relativo por métricas certificadas'
)
sunburst_fig.update_traces(
    hovertemplate=(
        '<b>%{label}</b><br>'
        'Valor: %{value:,.0f}<br>'
        'Participación en el nivel superior: %{percentParent:.1%}<br>'
        'Participación en el total: %{percentRoot:.1%}<extra></extra>'
    )
)
sunburst_fig.show()

## 8. Próximos pasos sugeridos
- Incorporar series temporales si se dispone de cortes históricos.
- Enriquecer `tourism_metrics.csv` con datos regionales adicionales para mapas coropléticos.
- Conectar este cuaderno a un dashboard (Dash o Streamlit) para compartirlo con stakeholders.